# 📁 New Repository Structure

**Configuration files have been moved to be more accessible:**

- **Tool Config**: `config/tools.yaml` (was `src/config/tools/config.yaml`)
- **MCP Servers**: `config/mcp_servers/*.json` (was `src/mcp_integration/servers/*.json`)

**Benefits:**
- ✅ Easy to find at project root level
- ✅ Clear separation of config vs code
- ✅ Auto-discovery still works

**To view available tools:**
```bash
python -m src.config.tools.registry
```

---


# WhatsApp HR Assistant - Complete Pipeline Testing

This notebook provides comprehensive testing for the entire WhatsApp HR Assistant pipeline.

## What This Notebook Tests

1. **Tool Discovery System**
   - Auto-discovery of internal MCP tools
   - Auto-discovery of external MCP configurations
   - Tool inspection and validation

2. **Tool Loading**
   - Dynamic tool loading from YAML config
   - Internal MCP tools
   - External MCP client tools
   - Tool switching (internal vs external)

3. **Agent Pipeline**
   - Tool integration with LangGraph agent
   - End-to-end workflow testing
   - Error handling

## Setup

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Set up logging
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print(f"✅ Project root: {project_root}")
print(f"✅ Python path configured")

## Part 1: Tool Discovery Testing

Test the auto-discovery system for tools.

In [ ]:
from src.tools import ToolDiscovery
from src.tools import ToolInspector

print("="*80)
print("TOOL DISCOVERY TEST")
print("="*80)

# Create discovery instance
discovery = ToolDiscovery()

# Test 1: Discover internal MCP tools
print("\n1️⃣ Testing Internal MCP Tool Discovery")
print("-" * 80)
internal_tools = discovery.discover_internal_mcp_tools()

print(f"\n📊 Found {len(internal_tools)} internal tool groups:")
for tool_name, tool_path in sorted(internal_tools.items()):
    if isinstance(tool_path, list):
        print(f"  • {tool_name}: {len(tool_path)} tools")
        for path in tool_path:
            print(f"    - {path.split('.')[-1]}")
    else:
        print(f"  • {tool_name}: {tool_path.split('.')[-1]}")

# Test 2: Discover MCP configurations
print("\n2️⃣ Testing MCP Configuration Discovery")
print("-" * 80)
mcp_configs = discovery.discover_mcp_configs()

print(f"\n📊 Found {len(mcp_configs)} MCP configurations:")
for config_name, config_data in sorted(mcp_configs.items()):
    config_type = config_data.get('type', 'unknown')
    print(f"  • {config_name}: {config_type}")
    if config_type == 'stdio' and 'command' in config_data:
        cmd = config_data['command']
        args = ' '.join(config_data.get('args', []))
        print(f"    Command: {cmd} {args}")
    elif 'url' in config_data:
        print(f"    URL: {config_data['url']}")

# Test 3: Get all available tools
print("\n3️⃣ Testing Complete Tool Discovery")
print("-" * 80)
all_tools = discovery.get_all_available_tools()

print("\n📊 Summary:")
summary = all_tools['summary']
print(f"  Internal MCP Tools: {summary['internal_mcp_count']} tools in {summary['internal_mcp_groups']} groups")
print(f"  MCP Configurations: {summary['mcp_configs_count']} configs")
print(f"  Total Sources: {summary['total_sources']}")

print("\n✅ Tool Discovery Tests Passed!")

## Part 2: Tool Inspector Testing

Test the tool inspection and validation utilities.

In [ ]:
print("="*80)
print("TOOL INSPECTOR TEST")
print("="*80)

inspector = ToolInspector()

# Test 1: List all tools in table format
print("\n1️⃣ Listing All Available Tools")
print("-" * 80)
print(inspector.list_all_tools(format="table"))

# Test 2: Get details for specific tools
print("\n2️⃣ Testing Tool Detail Inspection")
print("-" * 80)
test_tools = ['gmail', 'calendar', 'cv_processing']

for tool_name in test_tools:
    print(f"\n📦 {tool_name.upper()}:")
    details = inspector.get_tool_details(tool_name)
    if details:
        print(f"  Type: {details['type']}")
        if 'description' in details:
            desc = details['description'].split('\n')[0][:100]
            print(f"  Description: {desc}...")
        if 'tools' in details:
            print(f"  Contains {len(details['tools'])} sub-tools")
    else:
        print(f"  ⚠️  Tool not found")

# Test 3: Validate configuration
print("\n3️⃣ Validating Tool Configuration")
print("-" * 80)
validation = inspector.validate_tool_config()

print(f"\n📊 Validation Results:")
print(f"  Valid: {'✅ Yes' if validation['valid'] else '❌ No'}")
print(f"  Configured Tools: {validation['tool_count']}")
print(f"  Enabled Tools: {validation['enabled_count']}")

if validation['issues']:
    print("\n❌ Issues Found:")
    for issue in validation['issues']:
        print(f"  • {issue}")
else:
    print("\n✅ No issues found!")

if validation['warnings']:
    print("\n⚠️  Warnings:")
    for warning in validation['warnings'][:5]:  # Show first 5
        print(f"  • {warning}")
    if len(validation['warnings']) > 5:
        print(f"  ... and {len(validation['warnings']) - 5} more")

print("\n✅ Tool Inspector Tests Passed!")

## Part 3: Tool Loading Testing

Test the dynamic tool loading system.

In [ ]:
# Tool loader now uses config/tools.yaml (at project root)
# Tool loader now uses config/tools.yaml (at project root)
# Tool loader now uses config/tools.yaml (at project root)
# Tool loader now uses config/tools.yaml (at project root)
from src.tools import ToolLoader

print("="*80)
print("TOOL LOADING TEST")
print("="*80)

# Create loader instance
loader = ToolLoader()

# Test 1: List available tools before loading
print("\n1️⃣ Checking Available vs Configured Tools")
print("-" * 80)
available_info = loader.list_available_tools()

print("\n📊 Tool Status:")
print(f"  Available Internal: {available_info['summary']['internal_mcp_count']}")
print(f"  Available MCP Configs: {available_info['summary']['mcp_configs_count']}")
print(f"  Configured: {available_info['summary']['configured_count']}")
print(f"  Enabled: {available_info['summary']['enabled_count']}")

print("\n📋 Configured Tools:")
for name, cfg in sorted(available_info['configured'].items()):
    status = "✅" if cfg['enabled'] else "⏭️"
    print(f"  {status} {name}: {cfg['mode']}")

# Test 2: Load tools
print("\n2️⃣ Loading Tools Dynamically")
print("-" * 80)

try:
    tools = loader.get_tools()
    
    print(f"\n✅ Successfully loaded {len(tools)} tools")
    
    # Get summary
    summary = loader.get_tool_summary()
    
    print("\n📊 Loaded Tools:")
    for tool_info in summary['tools']:
        print(f"  • {tool_info['name']}")
        print(f"    {tool_info['description'][:80]}...")
    
    if summary['active_clients']:
        print(f"\n🌐 Active MCP Clients: {', '.join(summary['active_clients'])}")
    
except Exception as e:
    print(f"\n❌ Error loading tools: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ Tool Loading Tests Passed!")

## Part 4: Agent Integration Testing

Test tools integration with the LangGraph agent.

In [ ]:
print("="*80)
print("AGENT INTEGRATION TEST")
print("="*80)

# Test 1: Check tool factory integration
print("\n1️⃣ Testing Tool Factory Integration")
print("-" * 80)

try:
    from src.agents.tool_factory import get_tools
    from src.config import settings
    
    print(f"Current TOOL_MODE: {settings.TOOL_MODE}")
    
    factory_tools = get_tools()
    print(f"\n✅ Tool factory loaded {len(factory_tools)} tools")
    
    for tool in factory_tools:
        print(f"  • {tool.name}")
    
except Exception as e:
    print(f"\n❌ Error with tool factory: {e}")
    import traceback
    traceback.print_exc()

# Test 2: Create simple agent workflow
print("\n2️⃣ Testing Agent Workflow")
print("-" * 80)

try:
    from langchain_google_genai import ChatGoogleGenerativeAI
    from langgraph.prebuilt import create_react_agent
    
    # Create LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.7
    )
    
    # Bind tools
    llm_with_tools = llm.bind_tools(factory_tools)
    
    print("\n✅ Created LLM with tool bindings")
    print(f"   Model: gemini-1.5-flash")
    print(f"   Bound tools: {len(factory_tools)}")
    
except Exception as e:
    print(f"\n❌ Error creating agent: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ Agent Integration Tests Passed!")

## Part 5: Individual Tool Testing

Test specific tools to ensure they work correctly.

In [ ]:
print("="*80)
print("INDIVIDUAL TOOL TESTING")
print("="*80)

# Test internal MCP tools directly
print("\n1️⃣ Testing Internal MCP Tools")
print("-" * 80)

# Test DateTime tool
try:
    from src.tools.utilities.datetime_mcp import DateTimeMCPTool
    
    datetime_tool = DateTimeMCPTool()
    print(f"\n📅 DateTimeMCPTool:")
    print(f"  Name: {datetime_tool.get_name()}")
    
    # Test execution
    result = datetime_tool.execute(operation="get_current")
    print(f"  Result: {result[:100]}...")
    print("  ✅ DateTime tool works!")
    
except Exception as e:
    print(f"  ❌ Error: {e}")

# Test Gmail tool (without actual API call)
try:
    from src.tools.google.gmail_mcp import GmailMCPTool
    
    gmail_tool = GmailMCPTool()
    print(f"\n📧 GmailMCPTool:")
    print(f"  Name: {gmail_tool.get_name()}")
    print(f"  Schema operations: {gmail_tool.get_input_schema()['properties']['operation']['enum']}")
    print("  ✅ Gmail tool initialized!")
    
except Exception as e:
    print(f"  ❌ Error: {e}")

# Test Calendar tool
try:
    from src.tools.google.calendar_mcp import CalendarMCPTool
    
    calendar_tool = CalendarMCPTool()
    print(f"\n📅 CalendarMCPTool:")
    print(f"  Name: {calendar_tool.get_name()}")
    print(f"  Schema operations: {calendar_tool.get_input_schema()['properties']['operation']['enum']}")
    print("  ✅ Calendar tool initialized!")
    
except Exception as e:
    print(f"  ❌ Error: {e}")

print("\n✅ Individual Tool Tests Passed!")


## Part 6: Configuration Switching Test

Test switching tools between internal and external modes.

In [ ]:
print("="*80)
print("CONFIGURATION SWITCHING TEST")
print("="*80)

import yaml
from pathlib import Path

config_path = project_root / "src" / "config" / "config/tools.yaml"

# Load current config
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("\n📊 Current Tool Configuration:")
print("-" * 80)

tools_config = config.get('tools', {})

for tool_name, tool_cfg in sorted(tools_config.items()):
    if not tool_cfg.get('enabled', True):
        continue
    
    mode = tool_cfg.get('mode', 'unknown')
    
    status_icon = "✅"
    details = ""
    
    if mode == 'internal_mcp':
        details = "(internal)"
    elif mode == 'mcp_client':
        if 'mcp_config_file' in tool_cfg:
            details = f"(external: {tool_cfg['mcp_config_file']})"
        else:
            details = "(external: inline config)"
    
    print(f"  {status_icon} {tool_name}: {mode} {details}")

print("\n💡 To switch a tool:")
print("   1. Edit config/tools.yaml")
print("   2. Change 'mode' from 'internal_mcp' to 'mcp_client' (or vice versa)")
print("   3. For mcp_client, add 'mcp_config_file' or 'mcp_config'")
print("   4. Reload the ToolLoader")

print("\n✅ Configuration Switching Test Passed!")

## Part 7: Cleanup

Clean up any open connections.

In [ ]:
print("="*80)
print("CLEANUP")
print("="*80)

# Cleanup MCP client connections
try:
    if 'loader' in locals():
        import asyncio
        
        # Run cleanup
        try:
            loop = asyncio.get_running_loop()
            import nest_asyncio
            nest_asyncio.apply()
            loop.run_until_complete(loader.cleanup())
        except RuntimeError:
            asyncio.run(loader.cleanup())
        
        print("✅ Cleaned up MCP connections")
    else:
        print("ℹ️  No loader to clean up")
except Exception as e:
    print(f"⚠️  Cleanup warning: {e}")

print("\n✅ All cleanup complete!")

## Summary

This notebook tested:

✅ Tool Discovery System
- Auto-discovery of internal MCP tools
- Auto-discovery of external MCP configurations

✅ Tool Inspector
- Listing all available tools
- Getting detailed tool information
- Validating configuration

✅ Tool Loading
- Dynamic loading from YAML
- Internal MCP tools
- External MCP client tools

✅ Agent Integration
- Tool factory integration
- LLM tool binding

✅ Individual Tools
- DateTime, Gmail, Calendar tools

✅ Configuration Management
- Switching between modes
- Config file references

## Next Steps

1. Add more tools to `src/mcp_integration/tools/`
2. Create more MCP configurations in `mcp_configs/`
3. Test with real agent workflows
4. Monitor performance and optimize